In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from pyspark.sql import SparkSession
import pyspark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1558510634615_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
spark = SparkSession \
    .builder \
    .appName("assignment2") \
    .getOrCreate()

VBox()

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1558510634615_0001,pyspark,idle,Link,Link,✔


In [4]:
# Check the python version 
import platform
print(platform.python_version())

VBox()

3.6.8

## Stage One Wireless Reviews

In [5]:
rev_data = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
revs = spark.read.csv(rev_data,header=True,sep='\t').cache()

VBox()

In [6]:
revs.count()

VBox()

5069140

In [7]:
revs.select("customer_id").distinct().count()

VBox()

2075970

In [8]:
revs.select("product_id").distinct().count()

VBox()

297919

In [9]:
# top 10 users ranked by the number of reviews they publish
revs.groupBy("customer_id").count().orderBy("count", ascending=False).show(10)
# the largest number of reviews written for a single user is 3582

VBox()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   43430756| 3582|
|   18116317| 2987|
|   52287429| 2747|
|   52496677| 2650|
|   51110953| 2624|
|   19792742| 2495|
|   20018062| 2492|
|   50068216| 2379|
|   14539589| 2269|
|   50881246| 2104|
+-----------+-----+
only showing top 10 rows

In [10]:
customer_median = revs.groupBy("customer_id").count().orderBy("count", ascending=False).approxQuantile("count", [0.5], 0)
print(customer_median)

VBox()

[1.0]

In [11]:
# the top 10 products ranked by the number of reviews they have
revs.groupBy("product_id").count().orderBy("count", ascending=False).show(10)
# the largest number of reviews written for a single product is 3936

VBox()

+----------+-----+
|product_id|count|
+----------+-----+
|B00127RAJY| 4969|
|B0000AQS0F| 4967|
|B00G5G7K7O| 4439|
|B0002IQJ96| 4409|
|B00G5G7EXY| 4207|
|B003ZSJ212| 3641|
|B000X9FLKM| 3414|
|B000K8LV1O| 2989|
|B00N1JQ2UO| 2476|
|B00AMR5LZA| 2253|
+----------+-----+
only showing top 10 rows

In [12]:
# the median number of reviews a product has
product_median = revs.groupBy("product_id").count().orderBy("count", ascending=False).approxQuantile("count", [0.5], 0)
print(product_median)

VBox()

[3.0]



# Stage Two

In [13]:
import pyspark.sql.functions as F

# Segmentation is down using spark sql function split
rev_df_filtered = revs.withColumn("sentence", F.split(revs['review_body'], "\. |\? |\! ")).withColumn("sen_no", F.size("sentence")).filter("sen_no>1")

VBox()

In [14]:
from pyspark.sql.window import Window

# using window function to filter the products with less than or equal to 3 reviews

window_product = Window.partitionBy('product_id')
rev_df_filtered_1 = rev_df_filtered.withColumn('count', F.count('review_id').over(window_product)).filter('count>3').drop('count')

VBox()

In [15]:
# using window function to filter the users with less than or equal to 1 review

window_customer = Window.partitionBy('customer_id')
rev_df_filtered_2 = rev_df_filtered_1.withColumn('count', F.count('review_id').over(window_customer)).filter('count>1').drop('count')

VBox()

In [16]:
# top 10 users ranked by median number of sentences in the reviews they have published

magic_percentile = F.expr('percentile_approx(sen_no, 0.5)')

rev_df_filtered_2.groupBy('customer_id').agg(magic_percentile.alias('med_val')).orderBy("med_val", ascending=False).show(10)


VBox()

+-----------+-------+
|customer_id|med_val|
+-----------+-------+
|   39452841|    228|
|   24142448|    213|
|   51722321|    191|
|   32704480|    185|
|   13438601|    179|
|   18610538|    169|
|   47652132|    168|
|   24972624|    166|
|   46401180|    147|
|   26836040|    139|
+-----------+-------+
only showing top 10 rows

In [17]:
# top 10 products ranked by median number of sentences in the reviews they have received

rev_df_filtered_2.groupBy('product_id').agg(magic_percentile.alias('med_val')).orderBy("med_val", ascending=False).show(10)


VBox()

+----------+-------+
|product_id|med_val|
+----------+-------+
|B00RNTYI3G|    167|
|B00BGWDYNQ|    105|
|B002LMOCJA|    100|
|B003H5HGCU|     94|
|B001ELXT0W|     90|
|B0040SYTUU|     89|
|B0009GZG86|     86|
|B00DNF1RR4|     84|
|B00G6Q4TOK|     80|
|B005BYBZCC|     69|
+----------+-------+
only showing top 10 rows

In [18]:
# After filtering, the top 10 products by number of reviews is shown below:
rev_df_filtered_2.groupBy("product_id").count().orderBy("count", ascending=False).show(10)


VBox()

+----------+-----+
|product_id|count|
+----------+-----+
|B0000AQS0F| 2366|
|B00127RAJY| 1683|
|B00G5G7K7O| 1537|
|B003ZSJ212| 1475|
|B000X9FLKM| 1336|
|B00G5G7EXY| 1239|
|B00003CXCT| 1118|
|B00003CX5P|  995|
|B00006HBUJ|  991|
|B00AZMFK3K|  982|
+----------+-----+
only showing top 10 rows

# Stage 3 filtering for product id 

## Product chosen is B00G5G7EXY

## 5.1 Positive vs. Negative Reviews

In [19]:
# creating the positive dataframe and negative dataframe for the product chosen
# try this B00AMR5LZA
pos_df = rev_df_filtered_2.filter((F.col('product_id')=='B00AMR5LZA')).filter(F.col('star_rating')>=4)
neg_df = rev_df_filtered_2.filter((F.col('product_id')=='B00AMR5LZA')).filter(F.col('star_rating')<=2)

VBox()

In [20]:
# Create sentence 

sel = ['review_id', 'product_id', 'star_rating', 'sen_no', 'individual_sent']
pos_sen_df = pos_df.withColumn('individual_sent', F.explode('sentence')).select(sel).filter(F.length(F.col("individual_sent")) > 5).cache()
neg_sen_df = neg_df.withColumn('individual_sent', F.explode('sentence')).select(sel).filter(F.length(F.col("individual_sent")) > 5).cache()


VBox()

In [21]:
npos = pos_sen_df.count()
nneg = neg_sen_df.count()

VBox()

In [63]:
print(npos,nneg)

VBox()

2661 577

In [36]:
# Create pos and neg RDD with vectors of length 512

pos_rev_rdd = pos_sen_df.limit(npos).select('individual_sent').rdd.map(lambda row: str(row[0])).cache()
neg_rev_rdd = neg_sen_df.limit(nneg).select('individual_sent').rdd.map(lambda row: str(row[0])).cache()


VBox()

## RDD Version

In [37]:
def review_embed(rev_text_partition):
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
    embed = hub.Module(module_url)
    # mapPartition would supply element inside a partition using generator stype
    # this does not fit tensorflow stype
    rev_text_list = [text for text in rev_text_partition]
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(embed(rev_text_list))
    return message_embeddings


VBox()

In [38]:
def Cosine_distance(rdd):
    # rdd: ((a, ind_a), (b, ind_b))
    a = rdd[0][0]
    ia = rdd[0][1]
    b = rdd[1][0]
    ib = rdd[1][1]
    return ((ia, ib), 1-a@b/np.sqrt(a@a*(b@b)))

VBox()

In [39]:
# positive review  analysis

pos_embedding = pos_rev_rdd.mapPartitions(review_embed).zipWithUniqueId().cache()

pos_cartesian = pos_embedding.cartesian(pos_embedding).map(Cosine_distance).cache()

pos_sum = pos_cartesian.map(lambda x: (x[0][0], x[1])).reduceByKey(lambda a, b: a+b).cache()

pos_min = pos_sum.min(lambda x:x[1])

pos_avg_dist = pos_sum.values().sum()/(npos*(npos-1))

print('The average distance is:', pos_avg_dist)

VBox()

The average distance is: 0.7148459355281314
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 47152)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 266, in handle
    poll(authenticate_and_accum_updates)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 254, in au

In [40]:
# Top 10 nearest neighbours positive review

pos_centre = pos_cartesian.map(lambda x: (x[0][0], x[1])).filter(lambda x: x[0] == pos_min[0]).collect()

pos_centre_dist = [j for i,j in pos_centre]

pos_neighbour_ind = np.array(pos_centre_dist).argsort()[:11]

pos_review_id = pos_sen_df.select('review_id', 'individual_sent').rdd.collect()

print(pos_review_id[pos_neighbour_ind[0]][0], pos_neighbour_ind[0], pos_review_id[pos_neighbour_ind[0]][1])

for i in range(11):
    print(i, pos_review_id[pos_neighbour_ind[i]][0], pos_review_id[pos_neighbour_ind[i]][1])


VBox()

R3G3KHQ43Y860A 1081 This is the best bible movie ever made
0 R3G3KHQ43Y860A This is the best bible movie ever made
1 RPHGQC7A7NZHG One of the best made movie versions of the bible yet
2 RRX99ACI7D1QI Of all the Bible based films, this was my favorite Jesus
3 R1P0X6G9JNUSHN Truly a great movie about the BIBLE
4 R1NRPFNQIG56FX It is one of the Best Bible stories ever depicted on Movie setting if not The Best
5 R3NFXPKKG26EYT Of all of the Biblical movies I've seen, this by far is the best one yet
6 RC0FZ9SNACXIG Fantastic Movie for Bible lovers
7 R2IZTJLYM9G3XF This movie was so well made and so true to the actual bible
8 RWK2D9OTKPQIR This movie is excellent, and follows the Bible as it is written
9 R3T75PFCI57M6U This miniseries is the best by far of all the different movies made about God, Jesus, and the Bible
10 R1ZLGIJ7KWOA02 Very good adaptation of the Bible

## Negative review analysis

In [41]:

neg_embedding = neg_rev_rdd.mapPartitions(review_embed).zipWithIndex().cache()

neg_cartesian = neg_embedding.cartesian(neg_embedding).map(Cosine_distance).cache()

neg_sum = neg_cartesian.map(lambda x: (x[0][0], x[1])).reduceByKey(lambda a, b: a+b).cache()

neg_avg_dist = neg_sum.values().sum()/(nneg*(nneg-1))

print('The average distance is:', neg_avg_dist)


VBox()

The average distance is: 0.7222256875855203

In [42]:
# Top 10 nearest neighbours negative review

neg_min = neg_sum.min(lambda x:x[1])

neg_centre = neg_cartesian.map(lambda x: (x[0][0], x[1])).filter(lambda x: x[0] == neg_min[0]).collect()

neg_centre_dist = [j for i,j in neg_centre]
neg_neighbour_ind = np.array(neg_centre_dist).argsort()[:11]

neg_review_id = neg_sen_df.select('review_id', 'individual_sent').rdd.collect()

for i in range(11):
    print(i, neg_review_id[neg_neighbour_ind[i]][0], neg_review_id[neg_neighbour_ind[i]][1])


VBox()

0 R2SGR50XJ4JEW It didn't even follow much of what The Bible says, scenes added for drama
1 R3P1Y9W6AGITGH  The Bible often lacks detail, but this series takes great pains to remove even that amount of detail in the Bible in order to make room for cheesy action scenes
2 R1VUMSZI8PAMQL I noted several times that the script did not follow the Bible
3 R2TYF4KR0I67D5 They should have done the screenplay off a really accurate bible story book(s) and then went from there, filling in the lose ends(other parts that they wanted to add)with the bible
4 R1Y4QZVEI65WRH  There was too much time spent on the fight scenes and not enough on the real message of the Bible
5 R3MMPM137A6F11 In scene after scene important details were left out, or changed and it gets worst in the New Testament
6 R1VQPF4LSABJ9D It's like Quentin Tarantino directed this.<br /><br />Half of the series is completely fabricated (dialogue and scenes not in The Bible - beyond simply trying to create a bridge between stories) and 

# Stage 4 TF-IDF transformation

## positive reviews analysis

In [45]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF

def Cosine_distance(rdd):
    x = rdd[0][0]
    ia = rdd[0][1]
    y = rdd[1][0]
    ib = rdd[1][1]
    return ((ia, ib), 1-x.dot(y)/x.norm(2)/y.norm(2))

VBox()

In [46]:
# Positive review analysis

hashingTF = HashingTF()
pos_tf = hashingTF.transform(pos_rev_rdd).cache()
pos_idf = IDF(minDocFreq=1).fit(pos_tf)
pos_tfidf = pos_idf.transform(pos_tf).cache()

VBox()

In [47]:
pos_tfidf_id = pos_tfidf.zipWithUniqueId().cache()

pos_tfidf_cartesian = pos_tfidf_id.cartesian(pos_tfidf_id).map(Cosine_distance).cache()

pos_tfidf_sum = pos_tfidf_cartesian.map(lambda x: (x[0][0], x[1])).reduceByKey(lambda a, b: a+b).cache()

pos_tfidf_avg_dist = pos_tfidf_sum.values().sum()/(npos*(npos-1))

print('The average distance is:', pos_tfidf_avg_dist)


VBox()

The average distance is: 0.7569218811530731

In [48]:
# Top 10 nearest neighbours positive review

pos_tfidf_min = pos_tfidf_sum.min(lambda x:x[1])

pos_tfidf_centre = pos_tfidf_cartesian.map(lambda x: (x[0][0], x[1])).filter(lambda x: x[0] == pos_tfidf_min[0]).collect()

pos_centre_dist = [j for i,j in pos_tfidf_centre]

pos_neighbour_ind = np.array(pos_centre_dist).argsort()[:11]

for i in range(11):
    print(i, pos_review_id[pos_neighbour_ind[i]][0], pos_review_id[pos_neighbour_ind[i]][1])


VBox()

0 R3LFBWNMKXRA7G I found myself walking away from this with a better understanding of the magical mindset and a reconnection with who I used to be that suddenly provided me with an understanding of some of the poor choices I had made when I was a Christian, choices based on a belief that a supernatural being was in charge of everything that was happening around me and, generally, here on Earth, and that understanding helped me to be less hard on myself for making those poor choices because, as Maya Angelou would say, I did my best and when I knew better I did better.<br /><br />This reminder of where I came from also helped me to be more forgiving of my Christian friends who have a tendency not to care as much (and sometimes not at all) for non-Christian populations because they believe that their suffering is ordained by their Lord
1 R3LFBWNMKXRA7G I don't approve of their ideas and I don't like how they feel and I don't have as close a relationship with any of them as I have with fri

## Negative review analysis

In [49]:
# Negative review analysis

hashingTF = HashingTF()
neg_tf = hashingTF.transform(neg_rev_rdd).cache()
neg_idf = IDF(minDocFreq=1).fit(neg_tf)
neg_tfidf = neg_idf.transform(neg_tf).cache()

VBox()

In [50]:
neg_tfidf_id = neg_tfidf.zipWithUniqueId().cache()

neg_tfidf_cartesian = neg_tfidf_id.cartesian(neg_tfidf_id).map(Cosine_distance).cache()

neg_tfidf_sum = neg_tfidf_cartesian.map(lambda x: (x[0][0], x[1])).reduceByKey(lambda a, b: a+b).cache()

neg_tfidf_avg_dist = neg_tfidf_sum.values().sum()/(nneg*(nneg-1))

print('Average distance for TFIDF negative reviews:', neg_tfidf_avg_dist)


VBox()

Average distance for TFIDF negative reviews: 0.7600351790298421

In [51]:
# Top 10 nearest neighbours positive review review

neg_tfidf_min = neg_tfidf_sum.min(lambda x:x[1])

neg_tfidf_centre = neg_tfidf_cartesian.map(lambda x: (x[0][0], x[1])).filter(lambda x: x[0] == neg_tfidf_min[0]).collect()

neg_tfidf_centre_dist = [j for i,j in neg_tfidf_centre]

neg_tfidf_neighbour_ind = np.array(neg_tfidf_centre_dist).argsort()[:11]

print(neg_review_id[neg_tfidf_neighbour_ind[0]][0], neg_tfidf_neighbour_ind[0], neg_review_id[neg_tfidf_neighbour_ind[0]][1])

for i in range(11):
    print(i, neg_review_id[neg_tfidf_neighbour_ind[i]][0], neg_review_id[neg_tfidf_neighbour_ind[i]][1])


VBox()

R1LUM57F7DZUGH 229 Considering this is the history channel, all we're missing is space aliens.<br />The Bible miniseries is a densely vague series of happenings strung together by events which seemingly have no connection to one another offering no explanations, no context, no development, and no substance, with wooden characters that are all on screen for only ten minutes at a time before being dropped entirely for new characters that also seemingly have no connection to one another all begging for our sympathy even though we've got absolutely no time to get to know any of them or how they even fit into this massively dense vague piece of shclop written by people who have absolutely no understanding about what they're even writing about
0 R1LUM57F7DZUGH Considering this is the history channel, all we're missing is space aliens.<br />The Bible miniseries is a densely vague series of happenings strung together by events which seemingly have no connection to one another offering no expla

## Word2Vec Implementation of Stage 4
### positive class

In [72]:
from pyspark.sql.types import *
from pyspark.ml.feature import Word2Vec
pos_class_df = spark.createDataFrame(pos_rev_rdd.map(lambda x: [x]), ArrayType(StringType()))
word2vec = Word2Vec(vectorSize = 300, minCount=0, inputCol = "value", outputCol = "result")
model = word2vec.fit(pos_class_df)
pos_word2vec = model.transform(pos_class_df).select("result").rdd.map(lambda x:x[0]).cache()

VBox()

In [73]:
pos_w2v_id = pos_word2vec.zipWithUniqueId().cache()

pos_w2v_cartesian = pos_w2v_id.cartesian(pos_w2v_id).map(Cosine_distance).cache()

pos_w2v_sum = pos_w2v_cartesian.map(lambda x: (x[0][0], x[1])).reduceByKey(lambda a, b: a+b).cache()

pos_w2v_avg_dist = pos_w2v_sum.values().sum()/(npos*(npos-1))

print('The average distance is:', pos_w2v_avg_dist)


VBox()

The average distance is: 1.0000091232411779

In [74]:
# Top 10 nearest neighbours positive review

pos_w2v_min = pos_w2v_sum.min(lambda x:x[1])

pos_w2v_centre = pos_w2v_cartesian.map(lambda x: (x[0][0], x[1])).filter(lambda x: x[0] == pos_w2v_min[0]).collect()

pos_centre_dist = [j for i,j in pos_w2v_centre]

pos_neighbour_ind = np.array(pos_centre_dist).argsort()[:11]

for i in range(11):
    print(i, pos_review_id[pos_neighbour_ind[i]][0], pos_review_id[pos_neighbour_ind[i]][1])


VBox()

0 R1ZARVD12RP9W4  I was pleased to hear the viewership was higher than American Idol, so it must mean that young people were tuning in as well as adults
1 R2HV1N5425ARJG The actor that played Jesus did an excellent job
2 RZ3CYKWXQWYW5 Instead they covered all the really important and non-redundant stories.
3 R5TYTTKDJM9TL  I loved it so much intend to watch the series all over again
4 R9R1AQWHINDEE  Over all, it's a great tool to supplement the Bible study, and keep events connected and together,
5 R44Y4LQDT5UW8  I hope that those who saw President Obama's likeness portrayed as satan are mistaken
6 R2IZTJLYM9G3XF This movie was so well made and so true to the actual bible
7 R3Q42I4NNVS1M2  It was just wonderful
8 R2G3R7DXRQHFJI This is the lowest price I found for this series
9 R1CUIFTS7RFM6V  I would have added the sex and nudity in it because THAT was the actual politically incorrect STORY!<br /><br /> The Genesis details are left out, no history of Caine and Able, nor the third brot

### negative class

In [75]:
neg_class_df = spark.createDataFrame(neg_rev_rdd.map(lambda x: [x]), ArrayType(StringType()))
model = word2vec.fit(neg_class_df)
neg_word2vec = model.transform(neg_class_df).select("result").rdd.map(lambda x:x[0]).cache()

VBox()

In [76]:
neg_w2v_id = neg_word2vec.zipWithUniqueId().cache()

neg_w2v_cartesian = neg_w2v_id.cartesian(neg_w2v_id).map(Cosine_distance).cache()

neg_w2v_sum = neg_w2v_cartesian.map(lambda x: (x[0][0], x[1])).reduceByKey(lambda a, b: a+b).cache()

neg_w2v_avg_dist = neg_w2v_sum.values().sum()/(npos*(npos-1))

print('The average distance is:', pos_w2v_avg_dist)


VBox()

The average distance is: 1.0000091232411779

In [77]:
# Top 10 nearest neighbours positive review

neg_w2v_min = neg_w2v_sum.min(lambda x:x[1])

neg_w2v_centre = neg_w2v_cartesian.map(lambda x: (x[0][0], x[1])).filter(lambda x: x[0] == neg_w2v_min[0]).collect()

neg_centre_dist = [j for i,j in neg_w2v_centre]

neg_neighbour_ind = np.array(neg_centre_dist).argsort()[:11]

for i in range(11):
    print(i, neg_review_id[neg_neighbour_ind[i]][0], neg_review_id[neg_neighbour_ind[i]][1])


VBox()

0 R2EUQSBQF34SRS not Biblical account of Sodom & Gomorah with all the Samuri sword fighting
1 R33M0HGHD2EXCB  It's funny how everything out of Hollywood is distorted these days
2 R3DKZMFIALITNV I could hardly watch it because it was so bad
3 R1VQPF4LSABJ9D And what's with the sword play every ten minutes
4 R31UK8DHK4URJ2 But when they take Biblical fact and change it, is where I draw the line.<br />  Other things were also bothersome
5 R1WHFI3LUO0DYD If you are going to share the word with others, tell the truth and be accurate
6 RPXSCZ5NRR5HF  I am quite the opposite in that blood and sexuality cause me to turn a show off
7 RQ2T9RQW0RPE2  If the DVD or CD is something old and uninteresting to most, I have no problem with it.
8 R2NWEHJH3SBRYO And to think major pastors and bishops (holy men) sanctioned this movie?!?!
9 R3TPZWJIL4E8VR Jesus Christ Musical superstar the movie better for children there is also a broadway
10 R25LMUZBI74Q01 I'm very disappointed with both a production that 